In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip '/content/drive/MyDrive/stanford-parser-4.2.0.zip'
!pip install svgling

Archive:  /content/drive/MyDrive/stanford-parser-4.2.0.zip
replace stanford-parser-full-2020-11-17/ejml-simple-0.38.jar? [y]es, [n]o, [A]ll, [N]one, [r]ename: Traceback (most recent call last):

^C


In [ ]:
import os
from nltk.parse.stanford import StanfordParser
from nltk import ParentedTree, Tree
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import re

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
java_path = "/usr/lib/jvm/java-11-openjdk-amd64/bin/java"
os.environ['JAVAHOME'] = java_path

In [ ]:
sp = StanfordParser(path_to_jar='stanford-parser-full-2020-11-17/stanford-parser.jar',
                    path_to_models_jar='stanford-parser-full-2020-11-17/stanford-parser-4.2.0-models.jar')

<ipython-input-5-f7ffa9ee7245>:1: DeprecationWarning: The StanfordParser will be deprecated
Please use nltk.parse.corenlp.CoreNLPParser instead.
  sp = StanfordParser(path_to_jar='stanford-parser-full-2020-11-17/stanford-parser.jar',


In [ ]:
stopwords_set = set(['a', 'an', 'the', 'is','to','The','in','of','us'])

In [ ]:
def convert_isl(parsetree):
  dict = {}
  parenttree= ParentedTree.convert(parsetree)
  for sub in parenttree.subtrees():
    dict[sub.treeposition()]=0
  #"----------------------------------------------"
  isltree=Tree('ROOT',[])
  i=0
  for sub in parenttree.subtrees():
    if(sub.label()=="NP" and dict[sub.treeposition()]==0 and dict[sub.parent().treeposition()]==0):
      dict[sub.treeposition()]=1
      isltree.insert(i,sub)
      i=i+1

    if(sub.label()=="VP" or sub.label()=="PRP"):
      for sub2 in sub.subtrees():
        if((sub2.label()=="NP" or sub2.label()=='PRP') and dict[sub2.treeposition()]==0 and dict[sub2.parent().treeposition()]==0):
          dict[sub2.treeposition()]=1
          isltree.insert(i,sub2)
          i=i+1

  for sub in parenttree.subtrees():
    for sub2 in sub.subtrees():
      if(len(sub2.leaves())==1 and dict[sub2.treeposition()]==0 and dict[sub2.parent().treeposition()]==0):
          dict[sub2.treeposition()]=1
          isltree.insert(i,sub2)
          i=i+1

  return isltree

In [ ]:
def text_to_isl(sentence):
  pattern = r'[^\w\s]'

# Remove the punctuation marks from the sentence
  sentence = re.sub(pattern, '', sentence)
  englishtree=[tree for tree in sp.parse(sentence.split())]
  parsetree=englishtree[0]
  #print(parsetree)
  isl_tree = convert_isl(parsetree)
  words=parsetree.leaves()
  lemmatizer = WordNetLemmatizer()
  ps = PorterStemmer()
  lemmatized_words=[]
  for w in words:
    #w = ps.stem(w)
   lemmatized_words.append(lemmatizer.lemmatize(w))
   islsentence = ""
   for w in lemmatized_words:
    if w not in stopwords_set:
      islsentence+=w
      islsentence+=" "
  #islsentence = clean(words)
  islsentence = islsentence.lower()
  isltree=[tree for tree in sp.parse(islsentence.split())]
  return islsentence

In [ ]:
!pip install moviepy
!pip install pytube

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.4 MB/s eta 0:00:00


In [ ]:
from pytube import YouTube
def get_yt(link,path):
  yt = YouTube(link)
  yt.streams.filter(file_extension="mp4").get_by_resolution("360p").download(path)

In [ ]:
from moviepy.editor import *
import os

def cut_vid(filename,yt_name,start_min,start_sec,end_min,end_sec):
  clip = VideoFileClip(os.path.join(yt_path,yt_name+'.mp4'))
  clip1 = clip.subclip((start_min,start_sec),(end_min,end_sec))
  clip1.write_videofile(os.path.join('/content/NLP_dataset',filename+'.mp4'),codec='libx264')

In [ ]:
!mkdir NLP_dataset
!mkdir yt

mkdir: cannot create directory ‘NLP_dataset’: File exists
mkdir: cannot create directory ‘yt’: File exists


In [ ]:
root_path = '/content/NLP_dataset'
yt_path = '/content/yt'

In [ ]:
import pandas as pd

In [ ]:
NLP_videos = pd.read_csv('/content/drive/MyDrive/NLP_videos.csv')

In [ ]:
import pandas as pd
from IPython.display import HTML
from base64 import b64encode

In [ ]:
NLP_videos = pd.read_csv('/content/drive/MyDrive/NLP_videos.csv')
NLP_videos.head()

,Name,yt_name,Link,start_min,start_sec,end_min,end_sec
0,she,she,https://youtu.be/BXw0xmFB8pQ,0,0,0,3
1,eat,eat,https://youtu.be/d4nkMcBvUZM,0,0,0,3
2,time,time,https://youtu.be/4K-4FU5l7cY,0,0,0,3
3,day,day,https://youtu.be/11derBixy2I,0,0,0,2
4,her,her,https://youtu.be/qBNL8tfKk_8,0,0,0,2


In [ ]:
def text_to_vid(input_text):
  NLP_videos = pd.read_csv('/content/drive/MyDrive/NLP_videos.csv')
  root_path = '/content/NLP_dataset'
  yt_path = '/content/yt'
  videos = []
  clips=[]
  sentence = text_to_isl(input_text)
  print(sentence)
  words = sentence.split()
  for i in words:
    if (NLP_videos['Name'].eq(i)).any():
      idx = NLP_videos.index[NLP_videos['Name'] == i].tolist()
      get_yt(NLP_videos['Link'].iloc[idx[0]],yt_path)
      cut_vid(i,NLP_videos['yt_name'].iloc[idx[0]],NLP_videos['start_min'].iloc[idx[0]],NLP_videos['start_sec'].iloc[idx[0]],NLP_videos['end_min'].iloc[idx[0]],NLP_videos['end_sec'].iloc[idx[0]])
      videos.append(os.path.join(root_path,i+'.mp4'))
    else:
      for letter in i:
        idx = NLP_videos.index[NLP_videos['Name'] == letter].tolist()
        #print(letter,idx)
        get_yt(NLP_videos['Link'].iloc[idx[0]],yt_path)
        cut_vid(letter,NLP_videos['yt_name'].iloc[idx[0]],NLP_videos['start_min'].iloc[idx[0]],NLP_videos['start_sec'].iloc[idx[0]],NLP_videos['end_min'].iloc[idx[0]],NLP_videos['end_sec'].iloc[idx[0]])
        videos.append(os.path.join(root_path,letter+'.mp4'))
  for i in videos:
    clip = VideoFileClip(i)
    clips.append(clip)

  final = concatenate_videoclips(clips, method="compose")
  final.write_videofile("merged.mp4")

SPEECH TO TEXT

In [ ]:
!pip install pipwin
!pipwin install pyaudio


/usr/local/lib/python3.10/dist-packages/pipwin/command.py:66: UserWarning: Found a non Windows system. Package installation might not work.
  warn("Found a non Windows system. Package installation might not work.")
Building cache. Hang on . . .
Traceback (most recent call last):
  File "/usr/local/bin/pipwin", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/pipwin/command.py", line 84, in main
    cache = pipwin.PipwinCache()
  File "/usr/local/lib/python3.10/dist-packages/pipwin/pipwin.py", line 210, in __init__
    self.data = build_cache()
  File "/usr/local/lib/python3.10/dist-packages/pipwin/pipwin.py", line 103, in build_cache
    dl_function = re.search(r"function dl.*\}", soup.find("script").string).group(0)
AttributeError: 'NoneType' object has no attribute 'string'


In [ ]:
!pip install SpeechRecognition
!pip install ffmpeg-python

In [ ]:
# Ensure yt-dlp is installed
!pip install yt-dlp --upgrade

import speech_recognition as sr
from yt_dlp import YoutubeDL

def get_yt(link, path):
    # yt-dlp options for downloading
    ydl_opts = {
        'outtmpl': f'{path}/%(title)s.%(ext)s',  # Save the file in the specified path with the video title
        'format': 'bestvideo+bestaudio/best',   # Download best video and audio quality
        'merge_output_format': 'mp4',           # Merge video and audio to mp4 if necessary
    }

    with YoutubeDL(ydl_opts) as ydl:
        ydl.download([link])


In [ ]:

import speech_recognition as sr
r = sr.Recognizer()

In [ ]:
AUDIO_FILE = ("/content/NLP_test_coffee.wav")

with sr.AudioFile(AUDIO_FILE) as source:
    #reads the audio file. Here we use record instead of
    #listen
    audio = r.record(source)
speech=r.recognize_google(audio)

In [ ]:
print(speech)

I like drinking coffee


In [ ]:
text_to_vid(speech)

i like drinking coffee 
[youtube] Extracting URL: https://youtu.be/f7XWdoOSSNw
[youtube] f7XWdoOSSNw: Downloading webpage
[youtube] f7XWdoOSSNw: Downloading ios player API JSON
[youtube] f7XWdoOSSNw: Downloading mweb player API JSON
[youtube] f7XWdoOSSNw: Downloading player 03dbdfab
[youtube] f7XWdoOSSNw: Downloading m3u8 information
[info] f7XWdoOSSNw: Downloading 1 format(s): 135+140
[download] Destination: /content/yt/me.f135.mp4
[download] 100% of  158.97KiB in 00:00:01 at 106.65KiB/s 
[download] Destination: /content/yt/me.f140.m4a
[download] 100% of   88.77KiB in 00:00:00 at 212.89KiB/s 
[Merger] Merging formats into "/content/yt/me.mp4"
Deleting original file /content/yt/me.f140.m4a (pass -k to keep)
Deleting original file /content/yt/me.f135.mp4 (pass -k to keep)
Moviepy - Building video /content/NLP_dataset/i.mp4.
MoviePy - Writing audio in iTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /content/NLP_dataset/i.mp4



Moviepy - Done !
Moviepy - video ready /content/NLP_dataset/i.mp4
[youtube] Extracting URL: https://youtu.be/FCMh8YKvD-8
[youtube] FCMh8YKvD-8: Downloading webpage
[youtube] FCMh8YKvD-8: Downloading ios player API JSON
[youtube] FCMh8YKvD-8: Downloading mweb player API JSON
[youtube] FCMh8YKvD-8: Downloading m3u8 information
[info] FCMh8YKvD-8: Downloading 1 format(s): 135+140
[download] Destination: /content/yt/like.f135.mp4
[download] 100% of  228.89KiB in 00:00:00 at 388.81KiB/s 
[download] Destination: /content/yt/like.f140.m4a
[download] 100% of  101.61KiB in 00:00:00 at 416.78KiB/s 
[Merger] Merging formats into "/content/yt/like.mp4"
Deleting original file /content/yt/like.f140.m4a (pass -k to keep)
Deleting original file /content/yt/like.f135.mp4 (pass -k to keep)
Moviepy - Building video /content/NLP_dataset/like.mp4.
MoviePy - Writing audio in likeTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /content/NLP_dataset/like.mp4



Moviepy - Done !
Moviepy - video ready /content/NLP_dataset/like.mp4
[youtube] Extracting URL: https://youtu.be/VlqddxqAYQo
[youtube] VlqddxqAYQo: Downloading webpage
[youtube] VlqddxqAYQo: Downloading ios player API JSON
[youtube] VlqddxqAYQo: Downloading mweb player API JSON
[youtube] VlqddxqAYQo: Downloading player 03dbdfab
[youtube] VlqddxqAYQo: Downloading m3u8 information
[info] VlqddxqAYQo: Downloading 1 format(s): 135+140
[download] Destination: /content/yt/double handed.f135.mp4
[download] 100% of    4.52MiB in 00:00:01 at 3.09MiB/s   
[download] Destination: /content/yt/double handed.f140.m4a
[download] 100% of    1.24MiB in 00:00:00 at 4.65MiB/s   
[Merger] Merging formats into "/content/yt/double handed.mp4"
Deleting original file /content/yt/double handed.f135.mp4 (pass -k to keep)
Deleting original file /content/yt/double handed.f140.m4a (pass -k to keep)
Moviepy - Building video /content/NLP_dataset/d.mp4.
MoviePy - Writing audio in dTEMP_MPY_wvf_snd.mp3


...

MoviePy - Done.
Moviepy - Writing video /content/NLP_dataset/e.mp4



Moviepy - Done !
Moviepy - video ready /content/NLP_dataset/e.mp4
Moviepy - Building video merged.mp4.
MoviePy - Writing audio in mergedTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video merged.mp4



t:   6%|▌         | 44/742 [00:00<00:03, 212.02it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/NLP_dataset/i.mp4, 921600 bytes wanted but 0 bytes read,at frame 50/52, at time 2.00/2.04 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

t:  20%|██        | 149/742 [00:01<00:06, 95.94it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/NLP_dataset/like.mp4, 921600 bytes wanted but 0 bytes read,at frame 75/76, at time 3.00/3.03 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

t:  24%|██▍       | 177/742 [00:01<00:06, 83.18it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/NLP_dataset/d.mp4, 

Moviepy - Done !
Moviepy - video ready merged.mp4
